# Pilot Patients
## Inser or Upsert documents to Elastic search

In [1]:
es_base_url = 'http://localhost:9200'
headers = {'Content-Type': 'application/json'}

In [2]:
# auto sugestion with tab
%config IPCompleter.greedy=True
%config InlineBackend.figure_format = 'retina'
%matplotlib inline 

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import datetime as datetime

import requests
import json

import os
os.getcwd()

'/lhome/amardj/ISM/python_code/es_scripts/pilot_scripts'

In [4]:
sys_date = datetime.datetime.today().strftime( '%Y%m%d')
sys_date = f'{sys_date}' 
sys_date

'20201026'

In [5]:
data_dir = f'./data/{sys_date}/'

In [6]:
file_name = f'{data_dir}2_sp_dataset_clean.csv'

In [7]:
df_csv = pd.read_csv(file_name, index_col='id')

In [8]:
df_csv.head()

,activity_1,age_1,back_nevro_pow_1,back_nevro_reflex_1,back_nevro_sens_1,back_slr_1,birth_year,bmi_1,body_main_1,childhood_1,...,walk_aid_1,weight_1,work_ability_1,work_ability_3,work_ability_w2,work_ability_w4,work_ability_w8,work_type_1,clinic_id,patient_id
id,,,,,,,,,,,,,,,,,,,,,
0,quite reduced,59.0,NaN,NaN,NaN,NaN,1961,24.977043,NaN,very good,...,no walkaid,68.0,7,NaN,NaN,NaN,NaN,work demands much walking and lifting,support27,support27_1005_1961
1,slightly reduced,63.0,NaN,NaN,NaN,NaN,1957,NaN,NaN,good,...,no walkaid,NaN,6,NaN,NaN,NaN,NaN,mostly seated,support27,support27_1006_1957
2,quite reduced,30.0,NaN,NaN,NaN,NaN,1990,19.151192,NaN,good,...,no walkaid,56.0,5,NaN,NaN,NaN,NaN,mostly seated,support42,support42_3002_1990


In [9]:
js = df_csv.to_json(orient='index')

In [10]:
df1 = df_csv.fillna( value="")

In [11]:
dict_csv = df1.to_dict( orient='index')

In [12]:
json_str = json.dumps( dict_csv, allow_nan=True, ensure_ascii=False)

In [13]:
#print(json_str)

In [14]:
import codecs
import json

with codecs.open(f'{data_dir}2_sp_dataset_clean.json', 'w', encoding='utf-8') as f:
    json.dump(dict_csv, f, ensure_ascii=False)

In [15]:
import datetime
datetime.datetime.now().astimezone().isoformat()

'2020-10-26T09:54:15.626026+01:00'

In [16]:
es_main_index = 'manual_patients'
es_trace_index = 'manual_patient_traces'

# 

In [17]:
def get_id_for_pid( pid, es_index):
    params = (('filter_path', 'hits.hits._source'))
    data = '{  "query": {    "match": {      "patientId": "'+pid+'"    }  }}'
    response = requests.get(f'{es_base_url}/{es_index}/_search?filter_path=hits.hits._id', headers=headers, data=data)
    response_json = response.json()
    return response_json.get('hits', {}).get('hits', [{}])[0].get('_id')

In [18]:
def get_created_date_for_pid( pid, es_index):
    params = (('filter_path', 'hits.hits._source'))
    data = '{  "query": {    "match": {      "patientId": "'+pid+'"    }  }}'
    response = requests.get(f'{es_base_url}/{es_index}/_search?filter_path=hits.hits._source.dateCreated', headers=headers, data=data)
    response_json = response.json()
    return response_json.get('hits', {}).get('hits', [{}])[0].get('_source',{}).get('dateCreated')

In [19]:
get_created_date_for_pid('adam_saso_38888_1998', 'manual_patient_traces')

In [20]:
def insert_es_new_doc(json_doc, es_index):
    doc = json.dumps(json_doc)
    response = requests.post(f'{es_base_url}/{es_index}/_doc', headers=headers, data=doc)
    return response.json()

In [21]:
def update_es_doc_for_id( _id, json_doc, es_index):
    upsert_doc = json.dumps({'doc': json.dumps(json_doc), 'doc_as_upsert': True})
    response = requests.post(f'{es_base_url}/{es_index}/_update/{_id}?pretty', headers=headers, data=upsert_doc)
    return response.json()

In [22]:
def upadate_es_index(dict_csv, es_index):     
    
    es_trace_index = 'manual_patient_traces'
    
    for val in dict_csv:

        doc_json = {}
        
        patientDetails = dict_csv[val]

        patientId = patientDetails.get('patient_id')
        clinicId = patientDetails.get('clinic_id')

        current_date_time = datetime.datetime.now().astimezone().isoformat(timespec='milliseconds')

        doc_json.update(
            {
                "patientId" : patientId, 
                "clinicId" : clinicId, 
                "dateCreated" : current_date_time,
                "dateUpdated" : current_date_time, 
                "status" : "INITIAL", 
                "similarPatients" : { }, 
                "patientDetails": patientDetails
            }
        )
        
        #print(patientId)
        _id = get_id_for_pid( patientId, es_index)
        #print(_id)
        
        
        if _id is None:
            print('_id was None, thus inserting doc for pid', patientId)
                    
            insert_es_new_doc(doc_json, es_index)
            #insert_es_new_doc(doc_json, es_trace_index)
        else :
            print('Upserting doc for pid:', patientId, ' and _id: ', _id)
            
            doc_json['dateCreated'] = get_created_date_for_pid( patientId, es_index)
            
            insert_es_new_doc(doc_json, es_trace_index)
            update_es_doc_for_id(_id, doc_json, es_index)

In [23]:
upadate_es_index( dict_csv, es_main_index)

_id was None, thus inserting doc for pid support27_1005_1961
_id was None, thus inserting doc for pid support27_1006_1957
_id was None, thus inserting doc for pid support42_3002_1990
